In [1]:
import numpy as np
from load_dipa_water_nacl_training_set import load_training_set

### Load in a training set

_For details on how to construct your own training set, consult load_dipa_water_nacl_training_set.py as a template._

In [2]:
water_dipa_nacl, water_dipa, water_nacl = load_training_set(filepaths = [None, None, None, None]) 

Checking samples in mixture water_dipa
Checking samples in mixture water_nacl
Checking samples in mixture all_three
Checking samples in mixture None
Sample water1 has a duplicate!
Sample water2 has a duplicate!
Sample water3 has a duplicate!
Checking samples in mixture None
Checking samples in mixture water_dipa_nacl


In [3]:
water_dipa_nacl.chem_properties #  examine some chemical properties.

{'name': ['water', 'dipa', 'nacl'],
 'mw': [18.015, 101.19, 58.44],
 'nu': [1, 1, 2]}

Do a test-train split. This can be done randomly, or systematically. Here, we will do it systematically (non-randomly) using as a training set only DIPA-water and water-NaCl mixtures.

In [4]:
mix_train = water_dipa + water_nacl 

# filter the water-dipa-nacl set to only include samples that contain all three 
mix_test = water_dipa_nacl.filter({'water': [10 ** -5, 1], 'nacl': [10 ** -5, 1], 'dipa': [10 ** -5, 1]}) 
# Some print-outs are currently expected as the code checks for samples that exist in both datasets.

Checking samples in mixture None
Sample water1 has a duplicate!
Sample water2 has a duplicate!
Sample water3 has a duplicate!
Sample water1 has a duplicate!
Sample water2 has a duplicate!
Sample water3 has a duplicate!
Sample 5M has a duplicate!
Sample 5M_2 has a duplicate!
Sample 2M has a duplicate!
Sample 2M_2 has a duplicate!
Sample 4M has a duplicate!
Sample 4M_2 has a duplicate!
Sample a2 has a duplicate!
Sample a2a has a duplicate!
Sample a3 has a duplicate!
Sample a4 has a duplicate!
Checking samples in mixture None


In [5]:
from sklearn.metrics import mean_absolute_error

lbounds = [800, 2500] # set global bounds on your spectrum. 

# Set the number of windows to split your spectral range into.
# if nwindows = [1, 10], the code will first try to use the entire spectral range as a training set,
# then will split the range into 10 smaller ranges and try them sequentially.
nwindows = [10] 

sc = 'neg_mean_absolute_error' # scoring metric.
random_state = 42 # a replicable random state. 
tts_size = 0.25 # the fraction of samples to use as testing in the test-train split.
metric = mean_absolute_error # the scoring metric to use
metric_label = 'MAE' # for plotting and printouts

Create your search plan. We import GridSearchCV, an exhaustive search method that will try all combos of parameters until it finds the best-working fit. 

We will use Ridge regression, a type of machine learning regression.

The only adjustable parameter that we are telling the program to search over is the alpha parameter.

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

ridge_search_plan = GridSearchCV(
    Ridge(), {'alpha': np.logspace(-7, 7, 14)}, scoring=sc, cv=5
)

Create a list of search plans. Each is typically a different kind of machine learning model.

In [7]:
models = [ridge_search_plan] # currently, only one search plan is in the list. But you could include more.

Perform the search.

In [9]:
from mixture_composition_regression.cross_validation import cv_on_model_and_wavelength

viable_models, best_model, y, X = cv_on_model_and_wavelength(
    mix_train,
    nwindows, 
    models,
    target_chem='water',
    test_data=mix_test,
    tts_test_size=tts_size,
    tts_random_state=random_state,
    tolerance=5E-3,
    metric=metric,
    metric_label=metric_label,
    l_bounds=lbounds,
    plot_comparison=True,
    plot_comparison_savefile='./plots/axes_train'
)

in cv_and_wavel
Running analysis splitting interval into 10 windows.
Running analysis on Ridge()


ValueError: 
All the 70 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/ianbillinge/miniconda3/envs/mcr/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ianbillinge/miniconda3/envs/mcr/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py", line 1126, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/ianbillinge/miniconda3/envs/mcr/lib/python3.11/site-packages/sklearn/base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ianbillinge/miniconda3/envs/mcr/lib/py

In [ ]:
print(X)